# Projet d'exploration

Une grille sur laquelle les ennemis pourront se déplacer depuis un point d’entrée avec un but précis à atteindre. 
Les tours seront également placées sur cette grille et attaqueront les ennemis dans un rayon défini. 
Il y aura un système de gestion des ressources qui permettra à chaque côté de gagner de l’argent pour acheter ou améliorer son attaque/défense.

Lorsque l’ennemi sera en attaque, il utilisera l’algorithme A* pour lui permettre de trouver le chemin optimal jusqu’à la base adverse depuis son point de départ tout en évitant les obstacles.

Lorsque l’ennemi sera à la fois en défense ou en attaque, il utilisera le backtracking pour tester les différentes configurations possibles de placement de tours/de soldats afin de choisir la meilleure, en fonction de son budget.

## Algorithme de "Backtracking"

C’est un algorithme de résolution de problèmes qui consiste à trouver progressivement une solution en essayant différentes options et en les annulant si elles conduisent à une impasse. 

Il est récursif et est utilisé pour résoudre des problèmes en effectuant une série de choix. 
Si un choix mène à une impasse, l’algorithme revient au dernier choix valide effectué et essaie un chemin différent.

Dans le cas de notre jeu de tour de défense, cet algorithme nous sera très utile pour permettre au joueur de placer ses pions au bon endroit et de bien les choisir afin de gagner. 
Par exemple, si le joueur joue en défense, l’algorithme choisira pour lui quel type de tour est la meilleure pour survivre. 
Selon les ressources qu’il possède, le nombre d’ennemis, le nombre de tours déjà placées, le nombre de vagues d’ennemis, etc., l’algorithme proposera au joueur la meilleure tour, par exemple une tour de base, intermédiaire, avancée, hardcore, etc. 
Par la suite, le joueur aura le choix de l’emplacement de la tour dans la grille s’il le souhaite ; sinon, l’algorithme prendra en compte tous les facteurs mentionnés ci-dessus et décidera lui-même du meilleur emplacement pour la tour (mode automatique).

## Démo de l'algorithme

Dans cette démo, nous avons une grille de jeu simple. Le joueur est en défense pour pouvoir démontrer le mieux possible l'algorithme.
La base du joueur est en jaune, les tours en vertes, les ennemis en rouge et les cases noires sont des obstacles.

Nous avons un texte qui affiche les ressources du joueur afin d'avoir une idée de ce qu'il possède. 
Dans cette démo, les ressources augmentent de 15 à chaque 10 secondes pour simuler ce que le joueurs va gagné dans le vrai jeu lorsqu'il tuera les ennemis.
Chaque type de tour a un coût, une portée et un dommage infligé différent.
Pour le moment, il y a qu'un seul type de tour, la tour de base qui coute 30 ressources, qui a une porté de 1 et qui fait 10 de dommages.

Les ennemis sont placés au hasard pour simuler leurs mouvements.

Quand le joueur atteint une quantité suffisante de ressources, on lui permet de placer une tour automatiquement.
Le joueur clique sur le bouton "Place" et l'algorithme de backtracking commence.
Il vérfie d'abord qu'il n'y ai pas d'ennemis proche de la base dans une portée spécifique.
Par la suite, il vérifie combien d'ennemis la tour pourra toucher pour maximiser les dégats.
Pour trouver la meilleur position, on utilise des points. La position qui a le plus de points a la fin est celle qui a est la meilleure de le cas présent.

In [5]:
def is_tower_in_range(tower, x, y, player):
    """
    Check if a tower is within the range of another tower.
    This checks the surrounding cells based on the tower's range.
    """
    for other_tower, tx, ty in player.towers:
        if tower != other_tower:  # Don't check the tower itself
            # Check all cells within the tower's range (all cells around it)
            for dx in range(-(tower.range + 1), tower.range + 2):
                for dy in range(-(tower.range + 1), tower.range + 2):
                    if 0 <= tx + dx < cols and 0 <= ty + dy < rows:  # Ensure within bounds
                        # Check if (x, y) is in the range of this tower
                        if (tx + dx == x) and (ty + dy == y):
                            return True  # There's another tower within range
    return False

def is_enemy_near_base(enemies, base_x, base_y, base_range=5):
    """
    Check if any enemy is near the base (within a defined range).
    """
    for enemy in enemies:
        if enemy.alive and abs(enemy.x - base_x) <= base_range and abs(enemy.y - base_y) <= base_range:
            # Check all cells within the base's range (all cells around it)
            for dx in range(-(base_range + 1), base_range + 2):
                for dy in range(-(base_range + 1), base_range + 2):
                    if 0 <= base_x + dx < cols and 0 <= base_y + dy < rows:  # Ensure within bounds
                        if (base_x + dx == x) and (base_y + dy == y):
                            return True  # There's an enemy near the base
    return False

# Backtracking to find best tower placement
def backtrack_tower_placement(tower, enemies, player):
    best_position = None
    max_enemy_damage = 0
    
    for y in range(rows):
        for x in range(cols):
            if is_valid_move(x, y) and not is_tower_in_range(tower, x, y, player):
                tower.x, tower.y = x, y
                total_damage = 0
                
                # Calculate how many enemies this tower can hit
                for enemy in enemies:
                    if tower.is_in_range(enemy):
                        total_damage += 1  # Add 1 point for each enemy within range
                
                # Prioritize tower placement closer to the base if there are enemies nearby
                if is_enemy_near_base(enemies, base_x, base_y) and (abs(x - base_x) + abs(y - base_y)) <= 3:
                    total_damage += 50  # Bonus damage for towers close to base
                
                # Keep track of the best position (max damage)
                if total_damage > max_enemy_damage:
                    max_enemy_damage = total_damage
                    best_position = (x, y)

    return best_position

def main():
    player = Player(resources=20)   
    
    selected_type_tower = 0
    enemies = [Enemy(*generate_random_enemy(), 1, RED, 10) for _ in range(10)]
    
    last_resource_time = pygame.time.get_ticks()
    running = True
    
    while running:
        screen.fill(BLACK)

        # Draw grid and towers
        draw_grid(player, enemies)

        current_time = pygame.time.get_ticks()
        if current_time - last_resource_time >= 10000:  # 10 seconds passed
            player.gain_resources(15)
            last_resource_time = current_time
            print(player.resources)            
            
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
            if event.type == pygame.MOUSEBUTTONDOWN and popup_visible:
                mouse_pos = pygame.mouse.get_pos()
                close_button = show_tower_popup(player)

                if close_button.collidepoint(mouse_pos):
                    popup_visible = False
                    
                    selected_tower = Tower(
                        towers[selected_type_tower].tower_type,
                        towers[selected_type_tower].cost,
                        towers[selected_type_tower].range,
                        towers[selected_type_tower].power
                    )
                    
                    best_position = backtrack_tower_placement(selected_tower, enemies, player)
                    
                    if best_position:
                        player.place_tower(selected_tower, best_position[0], best_position[1])
                        print(f"Tower placed at {best_position}")
                        
        pygame.display.flip()

    pygame.quit()
    sys.exit()